In [4]:
import numpy as np
import tensorflow as tf

if __name__ == "__main__":
    sample_num = 5000
    item_num = 500
    sample_set = []
    
    for i in range(sample_num):
        visit_items_index = np.random.randint(low=0, high=item_num, size=[5])
        continuous_features_value = np.random.uniform(low=-5.0, high=5.0, size = [16])
        next_visit_item_index = np.random.randint(low=0, high=item_num)
        sample = [visit_items_index, continuous_features_value, next_visit_item_index]
        sample_set.append(sample)
    
    #print(sample_set)
    # 训练数据
    filename = "./data/train.tfrecords"
    writer = tf.python_io.TFRecordWriter(filename)
    for sample in sample_set:
        visit_items_index = sample[0]
        continuous_features_value = sample[1]
        next_visit_item_index = sample[2]
        example = tf.train.Example(features=tf.train.Features(
            feature={
                "visit_items_index":tf.train.Feature(int64_list=tf.train.Int64List(value=visit_items_index)),
                "continuous_features_value":tf.train.Feature(float_list=tf.train.FloatList(value=continuous_features_value)),
                "next_visit_item_index":tf.train.Feature(int64_list=tf.train.Int64List(value=[next_visit_item_index]))
            }
        ))
        writer.write(example.SerializeToString())
    writer.close
    
    # 评估数据，由于数据是随机生成的，所以评估数据从训练数据中获取
    filename = "./data/evalation.tfrecords"
    writer = tf.python_io.TFRecordWriter(filename)
    i = 0
    for sample in sample_set:
        if i % 10 ==0:
            visit_items_inex = sample[0]
            continuous_features_value = sample[1]
            next_visit_item_index = sample[2]
            example = tf.train.Example(features=tf.train.Features(
                feature={
                    "visit_items_index":tf.train.Feature(int64_list=tf.train.Int64List(value=visit_items_index)),
                    "continuous_features_value":tf.train.Feature(float_list=tf.train.FloatList(value=continuous_features_value)),
                    "next_visit_item_index":tf.train.Feature(int64_list=tf.train.Int64List(value=[next_visit_item_index]))
                }
            ))
            writer.write(example.SerializeToString())
        i = i + 1
    writer.close




In [8]:

def parse_fn(example):
    
    f = {
        "visit_items_index":tf.FixedLenFeature([5], tf.int64),
        "continuous_features_value":tf.FixedLenFeature([16],tf.float32),
        "next_visit_item_index":tf.FixedLenFeature([],tf.int64)
    }
    parsed = tf.parse_single_example(example,f)
    next_visit_item_index = parsed.pop("next_visit_item_index")
    return parsed, next_visit_item_index
    

    
files = tf.data.Dataset.list_files("./data/train.tfrecords")
dataset = files.apply(
    tf.contrib.data.parallel_interleave(
        lambda filename: tf.data.TFRecordDataset(filename),
        cycle_length=16
    )
)   
dataset = dataset.repeat(1).map(map_func=parse_fn, num_parallel_calls=16)
dataset = dataset.prefetch(64).batch(3)
iterator = dataset.make_one_shot_iterator()
features , label = iterator.get_next()

with tf.Session() as sess:
    for i in range(5):
        res1, res2 = sess.run([features, label])
        print("第{}批".format(i), end=" ")
        print("result1= ",res1)
        print("result2= ",res2)

    

第0批 result1=  {'continuous_features_value': array([[-2.3454483 , -4.1814303 ,  0.1500645 ,  3.7637715 ,  0.68554026,
         0.1855723 , -0.85582125,  0.7381248 ,  4.26442   , -0.8246727 ,
         2.561126  ,  4.1661415 ,  4.8583384 , -2.1614516 ,  4.5817685 ,
         4.961258  ],
       [ 4.2181244 ,  3.7592523 , -3.845831  , -0.3855235 , -4.3285894 ,
        -1.2480698 ,  2.3493867 ,  0.04192824, -1.7496421 , -2.6138124 ,
        -0.1831362 , -4.6035266 , -3.035082  , -0.16792537, -2.6745994 ,
         1.914487  ],
       [-3.9164913 ,  4.017446  ,  0.53705394,  3.2567225 , -3.719488  ,
         2.9297504 ,  2.5302134 ,  0.10876238, -2.4215887 , -4.8300157 ,
        -4.6176405 , -3.8073885 , -0.6118949 , -0.8034856 ,  3.0096893 ,
         1.0442649 ]], dtype=float32), 'visit_items_index': array([[204, 456,  57, 366, 466],
       [100, 314, 285, 326,  85],
       [ 41, 194, 312, 309, 415]], dtype=int64)}
result2=  [330 197 206]
第1批 result1=  {'continuous_features_value': array([[ 4